In [1]:
!pip install wandb -qqq
!apt install tree

     |████████████████████████████████| 1.9 MB 4.7 MB/s 
     |████████████████████████████████| 174 kB 82.9 MB/s 
     |████████████████████████████████| 184 kB 79.8 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 173 kB 96.9 MB/s 
     |████████████████████████████████| 168 kB 92.5 MB/s 
     |████████████████████████████████| 168 kB 98.6 MB/s 
     |████████████████████████████████| 166 kB 79.9 MB/s 
     |████████████████████████████████| 166 kB 96.9 MB/s 
     |████████████████████████████████| 162 kB 100.7 MB/s 
     |████████████████████████████████| 162 kB 103.5 MB/s 
     |████████████████████████████████| 158 kB 93.8 MB/s 
     |████████████████████████████████| 157 kB 98.0 MB/s 
     |████████████████████████████████| 157 kB 79.3 MB/s 
     |████████████████████████████████| 157 kB 75.9 MB/s 
     |████████████████████████████████| 157 kB 80.0 MB/s 
     |████████████████████████████████| 157 kB 76.9 MB/s 
     |█████████

In [ ]:
# !wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import random
import torch
import torchvision
from torch.utils.data import TensorDataset
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from math import sqrt
import math, time
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error
from torch.utils.data import DataLoader

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [4]:
#this block was only used to connect the google drive for colab implementations
#if you're working on your local, please skip this part

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
#please add sys path to reach the modules of the functions

import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks/DataCraft_notebook/Code files/One py for each application/')

## FUNCTIONS

In [6]:
#import cerated early stopping, LSTM model and sequence functions in py files

from earlyStopping import EarlyStopping
from LSTM_model import LSTM
from sequenceDataset import SequenceDataset

In [7]:
def split(df, SeqLen): #splitdataset
  split = [int(len(df)*0.8), int(len(df)*0.9)] #give split index for train, valid, test
  train = df[:split[0]]  #~%80
  val = df[split[0]-SeqLen:split[1]] # ~%10
  test = df[split[1]-SeqLen:]  # ~%10
  return train, val, test

In [8]:
#create evaluation function to get the MAPE and MSE scores of the train, test, validation dataset

def evaluation(split,scaled,Steps,model,loader,criterion,scaler):
  #calculate test loss, MSE and MAPE   
  loss=0    
  y_pred = torch.tensor([]).cuda()
  y=torch.tensor([]).cuda()
  y=torch.reshape(y, (-1,))
  y_pred=torch.reshape(y_pred, (-1,))
  model.eval()
  with torch.no_grad():
      for X, Y in loader:
          X=X.cuda()
          Y=Y.cuda()
          y = torch.cat((y, Y), 0)
          predict = model(X)
          
          predict=torch.reshape(predict, (-1,))
          y_pred = torch.cat((y_pred.cuda(), predict), 0)
          loss += criterion(y_pred, y).item()

  avg_loss = loss / Steps
  print("{} loss:".format(split),avg_loss)

  #reshape, concatenate, and apply inverse scaler transform to calculate the error values
  scaled_open=np.reshape(scaled[:,1], (-1,1))
  y_pred=np.reshape(y_pred.cpu().detach().numpy(), (-1,1))
  pred_open=np.concatenate((y_pred, scaled_open), axis=1)
  pred_open = scaler.inverse_transform(pred_open)
  price_open=np.concatenate((np.reshape(y.cpu().detach().numpy(), (-1,1)), scaled_open), axis=1)
  price_open = scaler.inverse_transform(price_open)

  mse_score = mean_squared_error(price_open[:,0], pred_open[:,0])
  print(split+' MSE: %.2f MSE' % (mse_score))

  mape_score = mean_absolute_percentage_error(price_open[:,0], pred_open[:,0])
  print(split+" MAPE:",mape_score)

  return (mape_score,mse_score)


## DATASET

The code user should indicate which dataset will be processed in this notebook. Because there are 3 different datasets belonging to the 3 different companies. These include the stock prices of Apple, Samsung, and Xiaomi companies.

And also each company has its own target MAPE test scores to reach business objectives which are given in the Project Charter Document. So, MAPE test goals are indicated to save the model that has reached this target value.

Don't forget business objectives MAPE test values should be at least %10 less than the baseline MAPE test results. So, the "mape_test_score_goal" values were detected according to the baseline results.

In [9]:
#initialize the variables
company = "Apple"
mape_test_score_goal = 0 #keep business objective

In [10]:
#get the datset
with wandb.init(project="DSS") as r:
         
        # ✔️ declare which artifact we'll be using
        if company == 'Apple':
          artifact = r.use_artifact('metu_datacraft/DSS/Apple:v2', type='Data')
          mape_test_score_goal = 0.014968 
        elif company == 'Samsung':
          artifact = r.use_artifact('metu_datacraft/DSS/Samsung:v2', type='Data')
          mape_test_score_goal = 0.0099
        elif company == 'Xiaomi':
          artifact = r.use_artifact('metu_datacraft/DSS/Xiaomi:v2', type='Data')
          mape_test_score_goal = 0.0205
        else:
          raise Exception("Sorry, please enter the company name correctly")

        table = artifact.get('weekday_data')
        dataset= {"Date": table.get_column("Date"),"Price":table.get_column("Price"),"Open":table.get_column("Open"),
                  "High":table.get_column("High"), "Low":table.get_column("Low") , "Volume": table.get_column("Volume"),
                  "Change":table.get_column("Change")}
        data = pd.DataFrame(dataset)

wandb: Currently logged in as: becemerce (metu_datacraft). Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


For only Apple dataset, the dataset had to be filtered to get high performance on the model prediction. This filtering issue was not applied on the other datasets belong to Samsung and Xiaomi. 

In [11]:
#the data was filtered to get the values of stock prices after the year 2020 to get more powerful model
#this block is valid only for Apple dataset
if company == 'Apple':
  data['Date'] = pd.to_datetime(data.Date)
  data = data[data['Date']>'2020-01-01']
  data.reset_index(inplace=True)
  data.drop(columns={'index'},inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [12]:
data

,Date,Price,Open,High,Low,Volume,Change
0,2020-01-02,75.09,74.06,75.15,73.80,135.65,2.29
1,2020-01-03,74.36,74.29,75.14,74.12,146.54,-0.97
2,2020-01-06,74.95,73.45,74.99,73.19,118.58,0.79
3,2020-01-07,74.60,74.96,75.22,74.37,111.51,-0.47
4,2020-01-08,75.80,74.29,76.11,74.29,132.36,1.61
...,...,...,...,...,...,...,...
733,2022-10-25,152.34,150.09,152.49,149.36,74.73,1.93
734,2022-10-26,149.35,150.96,151.99,148.04,87.74,-1.96
735,2022-10-27,144.80,148.07,149.05,144.13,95.29,-3.05
736,2022-10-28,155.74,148.20,157.50,147.82,163.87,7.56


## FINE TUNING

In this section, after the architecture tuning, the following steps were applied to the LSTM model for the hyperparameter tuning and model performance improvement;
* Xavier initiation
* Learning rate scheduler 
* Batch size tuning
* Sequence size tuning


In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#create sweep config for the fine tuning

sweep_config = {  #This sweep was created for a generalized purpose, and it will take a long time to conclude. When creating sweeps, parameters can be given discrete values to reduce run time.
    #for more information https://docs.wandb.ai/guides/sweeps/define-sweep-configuration
    'method': 'bayes', #grid, random, bayes
     'metric': {
      'name': 'average_train_loss',
      'goal': 'minimize'   
    },
    'metric': {
      'name': 'average_valid_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'seq_len': { # Small sequence lengths shift input to obtain the output. 
            'distribution': 'int_uniform',
            'min': 1,
            'max': 90
        },
        'batch_size': {
           'distribution': 'int_uniform',
             'min': 4,
             'max': 128
        },
        'hidden_size': {
            'values': [8]
        },
        'num_layers' :{
            'values': [1]
        },
        }
    }


In [ ]:
#create sweep id
sweep_id = wandb.sweep(sweep_config, project="DSS", entity="metu_datacraft")

Create sweep with ID: owq2i7p2
Sweep URL: https://wandb.ai/metu_datacraft/DSS/sweeps/owq2i7p2


In [ ]:
#give constant parameters for the fine tuning
#input and output dimension should be 'one'
input_dim = 1
output_dim = 1
n_epochs=500000

In [ ]:
def train(config=None):
  with wandb.init(config=config):
    config = wandb.config

    data['Date'] = pd.to_datetime(data.Date)

    SeqLen = config.seq_len

    #split dataset
    train, valid, test = split(data[["Price","Open"]],SeqLen)

    #scale the colums and values
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train) #fit the scaler by using train dataset
    test_scaled = scaler.transform(test) #scale the test dataset
    valid_scaled = scaler.transform(valid) #scale the validation daatset

    train_data = pd.DataFrame(train_scaled, columns = ['Price','Open'])
    test_data = pd.DataFrame(test_scaled, columns = ['Price','Open'])
    valid_data = pd.DataFrame(valid_scaled, columns = ['Price','Open'])

    #get sequences of the dataset
    train_dataset = SequenceDataset(
        train_data,
        target="Price",
        features=["Open"],
        sequence_length=SeqLen
    )


    test_dataset = SequenceDataset(
        test_data,
        target="Price",
        features=["Open"],
        sequence_length=SeqLen
    )

    valid_dataset = SequenceDataset(
        valid_data,
        target="Price",
        features=["Open"],
        sequence_length=SeqLen
    )

    #create the dataloader for the dataset to create batches
    trainloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    testloader = DataLoader(test_dataset,batch_size=config.batch_size,shuffle=False)
    validloader = DataLoader(valid_dataset,batch_size=config.batch_size,shuffle=False)

    #call model
    model = LSTM(input_dim=input_dim, hidden_dim=config.hidden_size, num_layers=config.num_layers,output_dim=output_dim).to(DEVICE)

    criterion = torch.nn.MSELoss()    # define criterion for loss: mean-squared error 
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3) #define optimizer

    #apply learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, threshold=0.0001, threshold_mode='abs')

    #use early stopping for tuning
    early_stopping = EarlyStopping(patience=20, delta=0.0001, verbose=False)

    #keep steps to calculate average loss values
    trainSteps = len(train_dataset) //config.batch_size
    testSteps = len(test_dataset)//config.batch_size
    validSteps = len(valid_dataset)//config.batch_size
    count=0
    for epoch in range(n_epochs):
      model.train()
      train_loss = 0
      
      for X, y in trainloader:
          X=X.to(DEVICE)
          y=y.to(DEVICE)
          output = model(X)
          output=torch.reshape(output, (-1,))
          loss = criterion(output, y) #calculate the loss
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          train_loss += loss.item() #calculate the total loss
            
      #check the validation loss to measure the training performance
      valid_loss=0
      model.eval()
      with torch.no_grad():
          for X, y in validloader:
              X=X.to(DEVICE)
              y=y.to(DEVICE)
              predict = model(X)
              predict=torch.reshape(predict, (-1,))
              valid_loss += criterion(predict, y).item()
              
      #calculate the loss values of train and validation        
      avg_train_loss = train_loss / trainSteps  
      avg_valid_loss = valid_loss / validSteps

      print("Epoch: %d, Train loss: %1.5f, Valid loss: %1.7f" % (epoch, avg_train_loss,avg_valid_loss))

      wandb.log({"Epoch":epoch, "average_train_loss": avg_train_loss,"average_valid_loss":avg_valid_loss})
      
      #step the learning rate scheduler 
      scheduler.step(avg_valid_loss)

      early_stopping(avg_valid_loss, model)

            
      if early_stopping.early_stop:
          print("Early stopping at epoch: ", epoch)
          break


    mape_test,mse_test=evaluation("Test",test_scaled,testSteps,model,testloader,criterion,scaler)
    mape_train,mse_train=evaluation("Train",train_scaled,testSteps,model,trainloader,criterion,scaler)
    mape_valid,mse_valid=evaluation("Valid",valid_scaled,testSteps,model,validloader,criterion,scaler)

  
    #log the scores
    wandb.log({"mse_test_score": mse_test, "mape_test_score": mape_test,"mse_train_score":mse_train,"mape_train_score":mape_train})

    #put if condition to save the model when the model performance reaches the business objective
    if mape_test < mape_test_score_goal: 
      print("HERE")
      torch.save(model.state_dict(), "/content/gdrive/My Drive/Colab Notebooks/DataCraft_notebook/model{company}{number}.pth".format(company=company,number=count))  #####LOCAL İCİN DÜZENLENECEK
      count+=1

wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 2pizc3ny with config:
wandb: 	batch_size: 19
wandb: 	hidden_size: 8
wandb: 	num_layers: 1
wandb: 	seq_len: 55


Epoch: 0, Train loss: 1.03740, Valid loss: 1.5311639
Epoch: 1, Train loss: 0.87680, Valid loss: 1.1697037
Epoch: 2, Train loss: 0.69022, Valid loss: 0.7424261
Epoch: 3, Train loss: 0.37455, Valid loss: 0.3867288
Epoch: 4, Train loss: 0.20647, Valid loss: 0.2245779
Epoch: 5, Train loss: 0.13772, Valid loss: 0.1669071
Epoch: 6, Train loss: 0.09999, Valid loss: 0.1165178
Epoch: 7, Train loss: 0.07200, Valid loss: 0.0830216
Epoch: 8, Train loss: 0.04971, Valid loss: 0.0601044
Epoch: 9, Train loss: 0.03652, Valid loss: 0.0478657
Epoch: 10, Train loss: 0.03459, Valid loss: 0.0417735
Epoch: 11, Train loss: 0.02277, Valid loss: 0.0410911
Epoch: 12, Train loss: 0.01942, Valid loss: 0.0425829
EarlyStopping counter: 1 out of 20
Epoch: 13, Train loss: 0.01791, Valid loss: 0.0408229
Epoch: 14, Train loss: 0.01662, Valid loss: 0.0417162
EarlyStopping counter: 1 out of 20
Epoch: 15, Train loss: 0.01599, Valid loss: 0.0409769
EarlyStopping counter: 2 out of 20
Epoch: 16, Train loss: 0.01691, Valid los

Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
average_train_loss,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
average_valid_loss,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mape_test_score,▁
mape_train_score,▁
mse_test_score,▁
mse_train_score,▁
Epoch,155
average_train_loss,0.00415
average_valid_loss,0.01044
mape_test_score,0.01557


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: r3dmh2my with config:
wandb: 	batch_size: 113
wandb: 	hidden_size: 8
wandb: 	num_layers: 1
wandb: 	seq_len: 15


wandb: ERROR Run r3dmh2my errored: ZeroDivisionError('float division by zero')
wandb: Agent Starting Run: aprik6hl with config:
wandb: 	batch_size: 60
wandb: 	hidden_size: 8
wandb: 	num_layers: 1
wandb: 	seq_len: 26


Epoch: 0, Train loss: 1.30563, Valid loss: 0.8923838
Epoch: 1, Train loss: 1.25210, Valid loss: 0.9002700
EarlyStopping counter: 1 out of 20
Epoch: 2, Train loss: 1.20955, Valid loss: 0.9043733
EarlyStopping counter: 2 out of 20
Epoch: 3, Train loss: 1.15560, Valid loss: 0.9090718
EarlyStopping counter: 3 out of 20
Epoch: 4, Train loss: 1.10008, Valid loss: 0.9029711
EarlyStopping counter: 4 out of 20
Epoch: 5, Train loss: 1.03519, Valid loss: 0.8963703
EarlyStopping counter: 5 out of 20
Epoch: 6, Train loss: 0.97080, Valid loss: 0.8795859
Epoch: 7, Train loss: 0.89625, Valid loss: 0.8583258
Epoch: 8, Train loss: 0.81463, Valid loss: 0.8231944
Epoch: 9, Train loss: 0.72819, Valid loss: 0.7720358
Epoch: 10, Train loss: 0.64133, Valid loss: 0.7033899
Epoch: 11, Train loss: 0.56245, Valid loss: 0.6312790
Epoch: 12, Train loss: 0.48912, Valid loss: 0.5561036
Epoch: 13, Train loss: 0.42281, Valid loss: 0.4882446
Epoch: 14, Train loss: 0.36459, Valid loss: 0.4244357
Epoch: 15, Train loss: 0.

Epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
average_train_loss,██▇▇▇▆▆▅▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
average_valid_loss,███████▇▇▆▅▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mape_test_score,▁
mape_train_score,▁
mse_test_score,▁
mse_train_score,▁
Epoch,47
average_train_loss,0.02153
average_valid_loss,0.11602
mape_test_score,0.03781


wandb: Agent Starting Run: 9x68x99o with config:
wandb: 	batch_size: 42
wandb: 	hidden_size: 8
wandb: 	num_layers: 1
wandb: 	seq_len: 8


Epoch: 0, Train loss: 1.04866, Valid loss: 1.2095445
Epoch: 1, Train loss: 0.94941, Valid loss: 1.1659660
Epoch: 2, Train loss: 0.82507, Valid loss: 1.0900398
Epoch: 3, Train loss: 0.75436, Valid loss: 0.9976888
Epoch: 4, Train loss: 0.59405, Valid loss: 0.8472529
Epoch: 5, Train loss: 0.50003, Valid loss: 0.7323056
Epoch: 6, Train loss: 0.39250, Valid loss: 0.5976945
Epoch: 7, Train loss: 0.30132, Valid loss: 0.4674412
Epoch: 8, Train loss: 0.19869, Valid loss: 0.3591417
Epoch: 9, Train loss: 0.17924, Valid loss: 0.2976911
Epoch: 10, Train loss: 0.13213, Valid loss: 0.2539073
Epoch: 11, Train loss: 0.11944, Valid loss: 0.2320869
Epoch: 12, Train loss: 0.11705, Valid loss: 0.2144900
Epoch: 13, Train loss: 0.10400, Valid loss: 0.2006203
Epoch: 14, Train loss: 0.10480, Valid loss: 0.1837989
Epoch: 15, Train loss: 0.08411, Valid loss: 0.1651007
Epoch: 16, Train loss: 0.07361, Valid loss: 0.1525893
Epoch: 17, Train loss: 0.07289, Valid loss: 0.1416921
Epoch: 18, Train loss: 0.06061, Valid 

wandb: Ctrl + C detected. Stopping sweep.
